# Data import

In [1]:
import sqlite3
conn = sqlite3.connect('C:/JupData/mxm_dataset.db')
print(conn)

In [2]:
res = conn.execute("SELECT * FROM sqlite_master WHERE type='table'")
print(res)

Metadata import

In [3]:
conn_tmdb = sqlite3.connect('C:/JupData/track_metadata.db')

In [4]:
res.fetchall()

[('table', 'words', 'words', 2, 'CREATE TABLE words (word TEXT PRIMARY KEY)'),
 ('table',
  'lyrics',
  'lyrics',
  4,
  'CREATE TABLE lyrics (track_id, mxm_tid INT, word TEXT, count INT, is_test INT, FOREIGN KEY(word) REFERENCES words(word))')]

# Playing around in sql

In [5]:
res = conn.execute("SELECT word FROM words")
len(res.fetchall())

5000

In [6]:
res = conn.execute("SELECT word FROM words WHERE ROWID BETWEEN 50 AND 60")
res.fetchall()

[('out',),
 ('down',),
 ('get',),
 ('she',),
 ('was',),
 ('see',),
 ('if',),
 ('got',),
 ('never',),
 ('from',),
 ('he',)]

In [7]:
res = conn.execute("SELECT track_id FROM lyrics WHERE word='countdown' ORDER BY RANDOM() LIMIT 1")

In [8]:
res.fetchall()

[]

In [9]:
res = conn_tmdb.execute("SELECT artist_name, title FROM songs WHERE track_id='TRCKLDK12903CEA5A9'")
res.fetchall()

[('Old Crow Medicine Show', 'My Good Gal')]

# Got sick of SQL and switched to pandas

<b> Import data - select quantity with the number after "limit"

In [90]:
#Import words data

import pandas as pd
df = pd.read_sql_query("select * from lyrics limit 10000000;", conn)

In [91]:
print(df.shape)
print(df.tail())
print(len(df['word'].value_counts()))

(10000000, 5)
                   track_id  mxm_tid   word  count  is_test
9999995  TRPJVET128F92DE5CE  4145998  gotta      8        0
9999996  TRPJVET128F92DE5CE  4145998   same      2        0
9999997  TRPJVET128F92DE5CE  4145998   blue      1        0
9999998  TRPJVET128F92DE5CE  4145998   real      2        0
9999999  TRPJVET128F92DE5CE  4145998  chanc      2        0
5000


In [92]:
#Import track information data to maps track id to title

df_meta=pd.read_sql_query("SELECT * from songs limit 10000000;", conn_tmdb)

In [93]:
print(df_meta.shape)
print(df_meta.tail(1))

(1000000, 14)
                  track_id                                title  \
999999  TRYYYVU12903CD01E3  Fernweh feat. Sektion Kuchikäschtli   

                   song_id     release           artist_id  \
999999  SOWXJXQ12AB0189F43  So Oder So  AR7PLM21187B990D08   

                                 artist_mbid artist_name   duration  \
999999  3af2b07e-c91c-4160-9bda-f0b9e3144ed3       Texta  295.07873   

        artist_familiarity  artist_hotttnesss  year  track_7digitalid  \
999999            0.552977           0.454869  2004           8486723   

        shs_perf  shs_work  
999999        -1         0  


In [94]:
df_titles=df_meta.merge(df, on='track_id', how='right')

In [95]:
#print(df_titles.shape, '\n', df_titles[df_titles['track_id']=='TRAAPKW128F428BC93'])

In [96]:
df_titles.drop(labels=['song_id', 'shs_perf', 'shs_work', 'mxm_tid'],axis=1, inplace=True)

<b> first genres dataset, didn't end up using it

In [97]:
#first genres dataset

df_genres=pd.read_csv('C:\JupData\msd-topMAGD-genreAssignment.cls', delimiter="\t", header=None)

In [98]:
df_genres.columns=['track_id','genre']
print(df_genres.head(1))

             track_id     genre
0  TRAAAAK128F9318786  Pop_Rock


In [99]:
print(df_genres[df_genres['genre']=='Country'])

                  track_id    genre
28      TRAABHC128F933A3F8  Country
93      TRAADXK12903CFAE92  Country
144     TRAAGIU128F931BA5B  Country
148     TRAAGMC128F4292D0F  Country
150     TRAAGNL128F4299BF1  Country
208     TRAAIAE128F42AC53D  Country
323     TRAAMKI128F428537A  Country
326     TRAAMRE128F4298E7D  Country
334     TRAAMXN128F148AD52  Country
343     TRAANEK128F427D2D5  Country
363     TRAANTY12903CAE21E  Country
372     TRAAOEH128F426A82F  Country
374     TRAAOFQ128F92E1E46  Country
391     TRAAORB128F930A97F  Country
394     TRAAORZ128F421CB5C  Country
407     TRAAPKW128F428BC93  Country
415     TRAAPVB12903CDD85D  Country
457     TRAARRM128F42BC439  Country
604     TRAAXTL128F4222D39  Country
624     TRAAYRA128F92E9CC7  Country
720     TRABCLV128F428C59F  Country
726     TRABCSP128F93089E8  Country
758     TRABEDC128F421AFC7  Country
905     TRABKKZ128F427D470  Country
943     TRABLSO128F14A4707  Country
997     TRABNTM128F42A2713  Country
1011    TRABOGP12903CC080D  

<b> second genre dataset, nicer (more merges) 

In [100]:
df_genres2=pd.read_csv('C:\JupData\msd_tagtraum_cd1.cls', delimiter="\t", skiprows=8, usecols=[0,1], header=None)

In [101]:
df_genres2.columns=['track_id', 'genre']
print(df_genres2.head())

             track_id       genre
0  TRAAAAW128F429D538         Rap
1  TRAAABD128F429CF47    Pop_Rock
2  TRAAAED128E0783FAB        Jazz
3  TRAAAEF128F4273421    Pop_Rock
4  TRAAAEM128F93347B9  Electronic


<b> merging genres data with titles and words data

In [102]:
df_merged=df_titles.merge(df_genres2, on='track_id', how='inner')

In [103]:
print(df_merged.shape)

(2795896, 15)


In [104]:
print(df_merged.head(1))

             track_id           title         release           artist_id  \
0  TRMMMNO128F93539AA  In The Journey  In The Journey  AR4TLW81187B99683D   

                            artist_mbid    artist_name   duration  \
0  0685ac4a-5cfc-408a-b391-903ea20e00bf  Martin Sexton  319.81669   

   artist_familiarity  artist_hotttnesss  year  track_7digitalid word  count  \
0            0.641198           0.448653  2001           5749967    i     30   

   is_test genre  
0        0  Folk  


In [105]:
#checking that we have enough country data, making sure the data looks ok

import numpy as np
df_merged['is_country'] = np.where(df_merged['genre'] ==  'Country', 1,0)

#df_merged['is_country']=0
#df_merged['is_country'][df_merged['genre']=='Country']=1

In [267]:
print(df_merged.genre.value_counts())

Pop_Rock         1961352
Rap               258306
Country           143398
RnB               117861
Electronic         84984
Folk               67849
Reggae             47652
Latin              43231
Jazz               33939
Blues              22210
International       7737
New Age             3750
Vocal               3627
Name: genre, dtype: int64


In [106]:
print(df_merged[df_merged['is_country']==1])

                   track_id           title        release  \
669      TRMMHIC128F92E8AE7         Rowboat      Unchained   
670      TRMMHIC128F92E8AE7         Rowboat      Unchained   
671      TRMMHIC128F92E8AE7         Rowboat      Unchained   
672      TRMMHIC128F92E8AE7         Rowboat      Unchained   
673      TRMMHIC128F92E8AE7         Rowboat      Unchained   
674      TRMMHIC128F92E8AE7         Rowboat      Unchained   
675      TRMMHIC128F92E8AE7         Rowboat      Unchained   
676      TRMMHIC128F92E8AE7         Rowboat      Unchained   
677      TRMMHIC128F92E8AE7         Rowboat      Unchained   
678      TRMMHIC128F92E8AE7         Rowboat      Unchained   
679      TRMMHIC128F92E8AE7         Rowboat      Unchained   
680      TRMMHIC128F92E8AE7         Rowboat      Unchained   
681      TRMMHIC128F92E8AE7         Rowboat      Unchained   
682      TRMMHIC128F92E8AE7         Rowboat      Unchained   
683      TRMMHIC128F92E8AE7         Rowboat      Unchained   
684     

In [268]:
#excluding pop rock
df_merged2=df_merged[df_merged.genre!='Pop_Rock']

In [320]:
df_model=df_merged[['track_id', 'is_country', 'word', 'count']]

# Sparse matrix representation - efficient

In [321]:
from scipy.sparse import csr_matrix

track_sorted=sorted(df_model.track_id.unique())
words_sorted=sorted(df_model.word.unique())

track_id_u = list(track_sorted)
word_u = list(words_sorted)

data = df_model['count'].tolist()
row = df_model.track_id.astype('category', categories=track_id_u).cat.codes
col = df_model.word.astype('category', categories=word_u).cat.codes
sparse_matrix = csr_matrix((data, (row, col)), shape=(len(track_id_u), len(word_u)))

In [322]:
x_train=np.array(sparse_matrix.todense())

In [323]:
print(x_train.shape)
print(x_train[2860])

(33780, 4998)
[0 0 0 ..., 0 0 0]


In [324]:
y_train=df_model.groupby('track_id', sort=False)['is_country'].agg('mean')
y_train=np.array(y_train)
print(y_train[2980:3000])

[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]


# Naive bayes

In [115]:
# Naive bayes 
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

model1=MultinomialNB()
model1.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [116]:
#Metrics
expected=y_train
predicted=model1.predict(x_train)
print('training data \n' + metrics.classification_report(expected, predicted))
matrix=metrics.confusion_matrix(y_train, predicted)
print(matrix)

training data 
             precision    recall  f1-score   support

          0       0.97      0.92      0.94     32194
          1       0.19      0.38      0.25      1586

avg / total       0.93      0.90      0.91     33780

[[29655  2539]
 [  987   599]]


# Extra data pre-processing for more expensive models
SVM, Logistic and RF take a while to run, and could be prone to overfitting. The code below only keeps the top "top" most popular words (change 1 cell down to implement this)

In [325]:
#Creates an array that sums up each column of x_train, sorts it, keeps the top 200 and subsets based on this
x_train_pd=pd.DataFrame(x_train)
x_train_sum=pd.DataFrame(x_train_pd.sum(axis=0))
print(x_train_sum.shape)
x_train_sum['idx']=range(len(x_train_sum))
x_train_sum.sort_values(by=0, axis=0, ascending=False, inplace=True)

(4998, 1)


In [326]:
#Change top and the code below to exclude some observations.
_from=0
_to=500
keep_columns=x_train_sum.iloc[_from:_to]['idx']
keep_columns_array=np.array(keep_columns)
x_train_subset=x_train_pd.iloc[:,keep_columns_array]
x_train2=np.array(x_train_subset)

Removing stop words (the stop words list is at the end of the notebook, for style)

In [327]:
#This significantly decreases our sample size :( hopefully it's all noise
df_model_nostop=df_model.loc[~df_model.word.isin(stop_words)]
df_model_nostop.shape

(1868436, 4)

In [328]:
from scipy.sparse import csr_matrix

track_sorted=sorted(df_model_nostop.track_id.unique())
words_sorted=sorted(df_model_nostop.word.unique())

track_id_u = list(track_sorted)
word_u = list(words_sorted)

data = df_model_nostop['count'].tolist()
row = df_model_nostop.track_id.astype('category', categories=track_id_u).cat.codes
col = df_model_nostop.word.astype('category', categories=word_u).cat.codes
sparse_matrix_nostop = csr_matrix((data, (row, col)), shape=(len(track_id_u), len(word_u)))

x_train_nostop=np.array(sparse_matrix_nostop.todense())
y_train_nostop=df_model_nostop.groupby('track_id', sort=False)['is_country'].agg('mean')
y_train_nostop=np.array(y_train_nostop)

I'm afraid of overfitting without stop words. Create a subset of the top non stop words (note the index is shorter because of removal of stop words)

In [329]:
x_train_nostop_pd=pd.DataFrame(x_train_nostop)
x_train_nostop_sum=pd.DataFrame(x_train_nostop_pd.sum(axis=0))
print(x_train_nostop_sum.shape)
x_train_nostop_sum['idx']=range(len(x_train_nostop_sum))
x_train_nostop_sum.sort_values(by=0, axis=0, ascending=False, inplace=True)

(4895, 1)


In [330]:
#Change top and the code below to exclude some observations.
_from=0
_to=1000
keep_columns=x_train_nostop_sum.iloc[_from:_to]['idx']
keep_columns_array=np.array(keep_columns)
x_train_nostop_subset=x_train_nostop_pd.iloc[:,keep_columns_array]
x_train_nostop2=np.array(x_train_nostop_subset)

In [331]:
print(x_train_nostop2.shape,y_train_nostop.shape)

(33778, 1000) (33778,)


Dataset with only stop words

In [332]:
#This significantly decreases our sample size :( hopefully it's all noise
df_model_stop=df_model.loc[df_model.word.isin(stop_words)]
df_model_stop.shape

(927460, 4)

In [333]:
from scipy.sparse import csr_matrix

track_sorted=sorted(df_model_stop.track_id.unique())
words_sorted=sorted(df_model_stop.word.unique())

track_id_u = list(track_sorted)
word_u = list(words_sorted)

data = df_model_stop['count'].tolist()
row = df_model_stop.track_id.astype('category', categories=track_id_u).cat.codes
col = df_model_stop.word.astype('category', categories=word_u).cat.codes
sparse_matrix_stop = csr_matrix((data, (row, col)), shape=(len(track_id_u), len(word_u)))

x_train_stop=np.array(sparse_matrix_stop.todense())
y_train_stop=df_model_stop.groupby('track_id', sort=False)['is_country'].agg('mean')
y_train_stop=np.array(y_train_stop)

Test/train split - choose data here. Options are:
- x_train (all data, top 5k words)
- x_train2 (all data, subset of words - see _from and _to 6 cells up)
- x_train_nostop (no stop words)
- x_train_nostop2 (no stop words, subset of words - see _from and _to 2 cells up)

In [334]:
# RUN THIS CELL TO CHANGE SAMPLE
x=x_train
y=y_train
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.3, random_state=0)

# Logistic

In [ ]:
#regularization

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

C:\Users\Baptiste\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int32 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [336]:
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression(class_weight='balanced', C=1/100)

In [318]:
model2=logit.fit(X_train_std,Y_train)

In [319]:
#Metrics test
expected=Y_test
predicted=model2.predict(X_test)
print('test data \n' + metrics.classification_report(expected, predicted))
matrix=metrics.confusion_matrix(Y_test, predicted)
print(matrix)

#Metrics train
expected=Y_train
predicted=model2.predict(X_train)
print('training data \n' + metrics.classification_report(expected, predicted))
matrix=metrics.confusion_matrix(Y_train, predicted)
print(matrix)

test data 
             precision    recall  f1-score   support

          0       0.81      0.97      0.88      2125
          1       0.16      0.02      0.04       490

avg / total       0.69      0.79      0.73      2615

[[2060   65]
 [ 478   12]]
training data 
             precision    recall  f1-score   support

          0       0.83      0.99      0.90      5004
          1       0.54      0.07      0.12      1096

avg / total       0.78      0.82      0.76      6100

[[4939   65]
 [1019   77]]


# Random forest

In [291]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(class_weight='balanced_subsample')

In [292]:
model3=rfc.fit(X_train, Y_train)

In [293]:
#Metrics test
expected=Y_test
predicted=model3.predict(X_test)
print('test data \n' + metrics.classification_report(expected, predicted))
matrix=metrics.confusion_matrix(Y_test, predicted)
print(matrix)

#Metrics train
expected=Y_train
predicted=model3.predict(X_train)
print('training data \n' + metrics.classification_report(expected, predicted))
matrix=metrics.confusion_matrix(Y_train, predicted)
print(matrix)

test data 
             precision    recall  f1-score   support

          0       0.81      0.99      0.89      2125
          1       0.10      0.00      0.01       490

avg / total       0.68      0.81      0.73      2615

[[2107   18]
 [ 488    2]]
training data 
             precision    recall  f1-score   support

          0       0.97      1.00      0.98      5004
          1       0.99      0.86      0.92      1096

avg / total       0.97      0.97      0.97      6100

[[4998    6]
 [ 155  941]]


# SVM

In [309]:
from sklearn import svm
svmlin=svm.LinearSVC(class_weight='balanced')

In [310]:
model4=svmlin.fit(X_train, Y_train)

In [311]:
#Metrics test
expected=Y_test
predicted=model4.predict(X_test)
print('test data \n' + metrics.classification_report(expected, predicted))
matrix=metrics.confusion_matrix(Y_test, predicted)
print(matrix)

#Metrics train
expected=Y_train
predicted=model4.predict(X_train)
print('training data \n' + metrics.classification_report(expected, predicted))
matrix=metrics.confusion_matrix(Y_train, predicted)
print(matrix)

test data 
             precision    recall  f1-score   support

          0       0.81      0.32      0.46      2125
          1       0.19      0.67      0.29       490

avg / total       0.69      0.39      0.43      2615

[[ 690 1435]
 [ 163  327]]
training data 
             precision    recall  f1-score   support

          0       0.89      0.36      0.51      5004
          1       0.22      0.80      0.34      1096

avg / total       0.77      0.44      0.48      6100

[[1810 3194]
 [ 219  877]]


# Close sql connection

In [ ]:
conn.close()
conn_tmdb.close()

# Sparse matrix inefficient
This is legacy code, it's ridiculously slow

In [ ]:
df_model_piv=df_model.pivot(index='track_id', columns='word', values='count')
print(df_model_piv.head(20))

In [ ]:
df_model_piv=df_model_piv.iloc[0:100000]
df_model_piv.replace(to_replace='Nan', value=0, inplace=True)
df_model_piv = df_model_piv.rename(columns={'word': 'track_id'})
print(df_model_piv.head(1))

In [ ]:
df_is_country=df_merged[['track_id', 'is_country']]
df_is_country=df_is_country.groupby(by='track_id', as_index=True).mean()
print(df_is_country[df_is_country.is_country==True])

In [ ]:
df_model_sparse=pd.merge(df_model_piv, df_is_country, left_index=True, right_index=True, how='left')

In [ ]:
#df_model_sparse.to_csv('C:/JupData/abc.csv')

In [ ]:
from sklearn import preprocessing
import numpy as np

y_train=np.array(df_model_sparse['is_country'])
df_model_sparse.drop('is_country', axis=1, inplace=True)

In [ ]:
df_model_sparse.drop('track_id', axis=1, inplace=True)

In [ ]:
x_train=np.array(df_model_sparse)

In [ ]:
print(x_train.shape, y_train.shape)

In [ ]:
print(x_train[2860], y_train[2980:3000])

# Stop words list

In [39]:
stop_words=["a",
"about",
"above",
"after",
"again",
"against",
"all",
"am",
"an",
"and",
"any",
"are",
"aren't",
"as",
"at",
"be",
"because",
"been",
"before",
"being",
"below",
"between",
"both",
"but",
"by",
"can't",
"cannot",
"could",
"couldn't",
"did",
"didn't",
"do",
"does",
"doesn't",
"doing",
"don't",
"down",
"during",
"each",
"few",
"for",
"from",
"further",
"had",
"hadn't",
"has",
"hasn't",
"have",
"haven't",
"having",
"he",
"he'd",
"he'll",
"he's",
"her",
"here",
"here's",
"hers",
"herself",
"him",
"himself",
"his",
"how",
"how's",
"i",
"i'd",
"i'll",
"i'm",
"i've",
"if",
"in",
"into",
"is",
"isn't",
"it",
"it's",
"its",
"itself",
"let's",
"me",
"more",
"most",
"mustn't",
"my",
"myself",
"no",
"nor",
"not",
"of",
"off",
"on",
"once",
"only",
"or",
"other",
"ought",
"our",
"ours",
"ourselves",
"out",
"over",
"own",
"same",
"shan't",
"she",
"she'd",
"she'll",
"she's",
"should",
"shouldn't",
"so",
"some",
"such",
"than",
"that",
"that's",
"the",
"their",
"theirs",
"them",
"themselves",
"then",
"there",
"there's",
"these",
"they",
"they'd",
"they'll",
"they're",
"they've",
"this",
"those",
"through",
"to",
"too",
"under",
"until",
"up",
"very",
"was",
"wasn't",
"we",
"we'd",
"we'll",
"we're",
"we've",
"were",
"weren't",
"what",
"what's",
"when",
"when's",
"where",
"where's",
"which",
"while",
"who",
"who's",
"whom",
"why",
"why's",
"with",
"won't",
"would",
"wouldn't",
"you",
"you'd",
"you'll",
"you're",
"you've",
"your",
"yours",
"yourself",
"yourselves",
]
